In [1]:
import salabim_plus as sim_plus
import salabim as sim
import multiplexer
import OPTX
import BFPD
import splitter
#import horn_antenna
#import antenna_kit
#import cover
import section_A
import section_B
#import section_C1 as section_C
#import section_D
#import section_E
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 1000

with open('data/output_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
    env = sim_plus.EnvironmentPlus(trace=out)
    
    # step 2: create the machines in the simulation environment
    # do there need to be things like assembly benches here?
    VSWR_CTI          =sim_plus.Machine(var_name='VSWR_CTI',        env=env)
    VSWR_CTI_DBG      =sim_plus.Machine(var_name='VSWR_CTI_DBG',     env=env)
    OPTX_CTI          =sim_plus.Machine(var_name='OPTX_CTI',         env=env)
    BFPD_CTI          =sim_plus.Machine(var_name='BFPD_CTI',         env=env)
    BFPD_CTI_DBG      =sim_plus.Machine(var_name='BFPD_CTI_DBG',     env=env)
#    VSWR_CTI2         =sim_plus.Machine(var_name='VSWR_CTI2',        env=env)
    # maybe make VSWR_CTI1 and VSWR_CTI2 into a machine group that 
    # goes by the name of VSWR_CTI
#    MOD_FEEDASSY_PAT  =sim_plus.Machine(var_name='MOD_FEEDASSY_PAT', env=env)
#    static_CTI        =sim_plus.Machine(var_name='STATIC_CTI',       env=env)
#    static_CTI_DBG    =sim_plus.Machine(var_name='STATIC_CTI_DBG',   env=env)
#    COND_EST          =sim_plus.Machine(var_name='COND_EST',         env=env)
#    RF_Fit_Test       =sim_plus.Machine(var_name='RF_FIT_TEST',      env=env)
#    RF_CFG            =sim_plus.Machine(var_name='RF_CFG',           env=env)
 
    assembly_bench_1  = sim_plus.Machine(var_name='assembly_bench_1', env=env)
    #how many assembly benches should we create?
    
    quality_bench_1   = sim_plus.Machine(var_name='quality_bench_1',  env=env)
    #how many quality benches should we create?
    
    forklift_1        = sim_plus.Machine(var_name='forklift_1', env=env)
    #make as many forklifts are there are production controllers. Make sure to add
    #them to the machine group below

    # step 3: make machine grouping for common processes
    assembly_bench = sim_plus.MachineGroup(var_name='assembly_bench', env=env,
                                           machines=[assembly_bench_1])
    quality_bench  = sim_plus.MachineGroup(var_name='quality_bench', env=env,
                                           machines=[quality_bench_1])
    forklift       = sim_plus.MachineGroup(var_name='forklift', env=env, 
                                           machines=[forklift_1])
    
    
    # step 4: create the workers in the simulation environment
    # how many workers actually are there??
    assembler                  =sim_plus.Worker(var_name='assembler',    env=env, capacity=1)
    technician                 =sim_plus.Worker(var_name='technician',   env=env, capacity=1)
    production_control         =sim_plus.Worker(var_name='production_control', env=env, capacity=1)
    qual_inspector             =sim_plus.Worker(var_name='qual_inspector',   env=env, capacity=1)
    
    
    # step 5: make the shift schedules for the workers
    
    shift_schedule = misc_tools.make_shifts(shift_duration=24*60, 
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    production_control_shift = sim_plus.ShiftController(worker=production_control,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    qual_inspector_shift = sim_plus.ShiftController(worker=qual_inspector,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    # step 6: point to the step creation function
    
    # if we pull data from the database, can we update the create_routing and pass in an argument of the 
    # arrays that we want them to pull from / reference?
    
    multiplexer_steps_func  = multiplexer.create_routing
    OPTX_steps_func         = OPTX.create_routing
    BFPD_steps_func         = BFPD.create_routing
    splitter_steps_func     = splitter.create_routing
#    horn_antenna_steps_func = horn_antenna.create_routing
#    cover_steps_func        = cover.create_routing
#    antenna_kit_steps_func  = antenna_kit.create_routing
    section_A_steps_func    = section_A.create_routing
    section_B_steps_func    = section_B.create_routing
#    section_C_steps_func    = section_C.create_routing
#    section_D_steps_func    = section_D.create_routing
#    section_E_steps_func    = section_E.create_routing


    # step 7: make all of the generators that determine when entites are made
    
    multiplexer_gen = sim_plus.EntityGenerator(var_name='multiplexer', 
                                         steps_func=multiplexer_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
#    antenna_kit_gen = sim_plus.EntityGenerator(var_name='antenna_kit', 
#                                         steps_func=antenna_kit_steps_func, 
#                                         env=env,
#                                         arrival_type='ordered')
    OPTX_gen = sim_plus.EntityGenerator(var_name='OPTX', 
                                         steps_func=OPTX_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    BFPD_gen = sim_plus.EntityGenerator(var_name='BFPD', 
                                         steps_func=BFPD_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    splitter_gen = sim_plus.EntityGenerator(var_name='splitter', 
                                         steps_func=splitter_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
#    cover_gen = sim_plus.EntityGenerator(var_name='cover', 
#                                         steps_func=cover_steps_func, 
#                                         env=env,
#                                         arrival_type='ordered')
#    horn_antenna_gen = sim_plus.EntityGenerator(var_name='horn_antenna', 
#                                         steps_func=horn_antenna_steps_func, 
#                                         env=env,
#                                         arrival_type='ordered')
#    
    section_A_gen = sim_plus.EntityGenerator(var_name='section_A', 
                                          steps_func=section_A_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=4)
    section_B_gen = sim_plus.EntityGenerator(var_name='section_B', 
                                          steps_func=section_B_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=3)
#    section_C_gen = sim_plus.EntityGenerator(var_name='section_C', 
#                                          steps_func=section_C_steps_func,
#                                          env=env,
#                                          arrival_type='inv_based', 
#                                          cut_queue=True, 
#                                          inv_level=0)
#    section_D_gen = sim_plus.EntityGenerator(var_name='section_D', 
#                                          steps_func=section_D_steps_func,
#                                          env=env,
#                                          arrival_type='inv_based', 
#                                          cut_queue=True, 
#                                          inv_level=0)
#    section_E_gen = sim_plus.EntityGenerator(var_name='section_E', 
#                                          steps_func=section_E_steps_func,
#                                          env=env,
#                                          arrival_type='inv_based', 
#                                          cut_queue=True, 
#                                          inv_level=0)
    
    # step 8: create all of the inventory locations in the simulation environment
    
    # multiplexer_storage  = sim_plus.Storage(var_name='multiplexer', env=env)
    # OPTX_storage         = sim_plus.Storage(var_name='OPTX', env=env)
    # BFPD_storage         = sim_plus.Storage(var_name='BFPD', env=env)
    # splitter_storage     = sim_plus.Storage(var_name='splitter', env=env)
    # horn_antenna_storage = sim_plus.Storage(var_name='horn_antenna', env=env)
    # antenna_kit_storage  = sim_plus.Storage(var_name='antenna_kit', env=env)
    # cover_storage        = sim_plus.Storage(var_name='cover', env=env)
    #FLT4_storage         = sim_plus.Storage(var_name='FLT4', env=env)
    
    section_B_storage = sim_plus.Storage(var_name='section_B',env=env)
    
    
    multiplexer_kanban  = sim_plus.Kanban(var_name='multiplexer',env=env,
                                        kanban_attr=multiplexer.create_kanban_attrs(env.objs))
#    antenna_kit_kanban  = sim_plus.Kanban(var_name='antenna_kit',env=env,
#                                        kanban_attr=antenna_kit.create_kanban_attrs(env.objs))
    OPTX_kanban         = sim_plus.Kanban(var_name='OPTX',env=env,
                                        kanban_attr=OPTX.create_kanban_attrs(env.objs))
    BFPD_kanban         = sim_plus.Kanban(var_name='BFPD',env=env,
                                        kanban_attr=BFPD.create_kanban_attrs(env.objs))
    splitter_kanban     = sim_plus.Kanban(var_name='splitter',env=env,
                                        kanban_attr=splitter.create_kanban_attrs(env.objs))
#    horn_antenna_kanban = sim_plus.Kanban(var_name='horn_antenna',env=env,
#                                        kanban_attr=horn_antenna.create_kanban_attrs(env.objs))
#    cover_kanban        = sim_plus.Kanban(var_name='cover',env=env,
#                                        kanban_attr=cover.create_kanban_attrs(env.objs))
    section_A_kanban    = sim_plus.Kanban(var_name='section_A',env=env,
                                        kanban_attr=section_A.create_kanban_attrs(env.objs))
#    section_B_kanban    = sim_plus.Kanban(var_name='section_B',env=env,
#                                        kanban_attr=section_B.create_kanban_attrs(env.objs))
#    section_C_kanban    = sim_plus.Kanban(var_name='section_C',env=env,
#                                        kanban_attr=section_C.create_kanban_attrs(env.objs))
#    section_D_kanban     = sim_plus.Kanban(var_name='section_D',env=env,
#                                        kanban_attr=section_D.create_kanban_attrs(env.objs))
    
    
    
    # step 9: connect any entity boms and main_exits to its entity generator
    
    section_A_gen.bom = section_A.get_bom(env=env.objs)
    section_B_gen.bom = section_B.get_bom(env=env.objs)
#    section_C_gen.bom = section_C.get_bom(env=env.objs)
#    section_D_gen.bom = section_D.get_bom(env=env.objs)
#    section_E_gen.bom = section_E.get_bom(env=env.objs)
    
    multiplexer_gen.main_exit  = multiplexer_kanban
    OPTX_gen.main_exit         = OPTX_kanban
    BFPD_gen.main_exit         = BFPD_kanban
    splitter_gen.main_exit     = splitter_kanban
#    horn_antenna_gen.main_exit = horn_antenna_kanban
#    cover_gen.main_exit        = cover_kanban
#    antenna_kit_gen.main_exit  = antenna_kit_kanban
    section_A_gen.main_exit    = section_A_kanban
#    section_B_gen.main_exit    = section_B_kanban
#    section_C_gen.main_exit    = section_C_kanban
#    section_D_gen.main_exit    = section_D_kanban

    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    production_control_shift.activate(process='work')
    qual_inspector_shift.activate(process='work')
    
    # activate components
    multiplexer_gen.activate(process='arrive')
    OPTX_gen.activate(process='arrive')
    BFPD_gen.activate(process='arrive')
    splitter_gen.activate(process='arrive')
#    horn_antenna_gen.activate(process='arrive')
#    cover_gen.activate(process='arrive')
#    antenna_kit_gen.activate(process='arrive') 
    section_A_gen.activate(process='arrive')
    section_B_gen.activate(process='arrive')
#    section_C_gen.activate(process='arrive')
#    section_D_gen.activate(process='arrive')
#    section_E_gen.activate(process='arrive')
    
    # step 11: execute the simulation
    env.run(till=runtime)
print("finished")

# VSWR_CTI.in_queue.print_statistics()
# VSWR_CTI_queue.print_statistics()

finished


In [5]:
import plotly.graph_objects as go
import numpy as np


def getarray(variable):
    # Creates an array for the occupancy value at each timestep for a given variable
    array = []
    for i in range(0,runtime):
        array.append(variable.occupancy(i))
    array = np.asarray(array)
    return array

assembler_array = getarray(assembler) *100
technician_array = getarray(technician) *100
production_control_array = getarray(production_control) *100
qual_inspector_array = getarray(qual_inspector) *100

# define a list of 1 to 1000 (each timestep)
x = np.arange(1,1000)

#Plotting histogram with the data
fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="avg", y=assembler_array, x=x, name="assembler"))
fig.add_trace(go.Histogram(histfunc="avg", y=technician_array, x=x, name="technician"))
fig.add_trace(go.Histogram(histfunc="avg", y=production_control_array, x=x, name="production_control"))
fig.add_trace(go.Histogram(histfunc="avg", y=qual_inspector_array, x=x, name="qual_inspector"))

fig.update_layout(
    plot_bgcolor='white',
    title_text='Resourse Utilization', # title of plot
    xaxis_title_text='Time (minutes)', # xaxis label
    yaxis_title_text='% Utilization', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
    ,yaxis=dict(showgrid=True, zeroline=True,gridcolor="LightGray",dtick=10)
#     showgrid=True

)
# fig.update_layout(xaxis_showgrid=False, 
#                   yaxis_showgrid=True,
#                   gridcolor='LightPink')
# fig.update_yaxes(nticks=20)
fig.show()

In [4]:
#Salabim plus output

import output_viewer

filepath = 'data/output_'+now+'.txt'
trace_df = output_viewer.get_trace_df(filepath)
state_df = output_viewer.get_state_df(filepath)

duration = datetime.timedelta(minutes=1000)
start_time = datetime.datetime.now()

machine_list = ['VSWR_CTI',
               'VSWR_CTI_DBG',
               'OPTX_CTI',
               'BFPD_CTI',
               'BFPD_CTI_DBG',
               'assembly_bench_1',
               'quality_bench_1',
                'forklift_1']

worker_list = ['assembler',
               'technician',
               'production_control',
               'qual_inspector']

worker_cap_dict = {'assembler':1,
               'technician':1,
               'production_control':1,
               'qual_inspector':1 }


machine_df = output_viewer.get_machine_state_df(state_df, start_time, duration, machine_list)
worker_df = output_viewer.get_worker_state_df(state_df, start_time, duration, worker_list)
entity_df = output_viewer.get_entity_state_df(state_df, start_time, duration)

output_viewer.plot_machine_timeline(machine_df)
output_viewer.plot_machine_utilization(machine_df, duration)



output_viewer.plot_worker_in_use_timeline(worker_df, worker_list)
output_viewer.plot_worker_utilization(worker_df, worker_cap_dict)


output_viewer.plot_entity_timeline(entity_df, height=800)
 

In [125]:
# # assembly_bench_1.queue.print_statistics()
# # quality_bench_1.queue.print_statistics()
# # forklift_1.queue.print_statistics()
# # assembler.print_statistics()
# print(technician.capacity.mean())
# print(technician.claimed_quantity.mean())
# print(technician.available_quantity.mean()) 
# print(technician.occupancy.mean())
# # for i in range(1,100):
# #     print(str(i*10) + ':')
# #     print(assembler.capacity(i*10))
# #     print(assembler.claimed_quantity(i*10))
# #     print(assembler.available_quantity(i*10))
# #     print(assembler.occupancy(i*10))
# # print(assembler.capacity(1001))
# print(assembler.occupancy.bin_duration(1,1000))
# print(type(assembler.occupancy))
# # print(assembler.occupancy(1,100))
# assembler_array = []
# technician_array= []
# for i in range(1,runtime):
#     assembler_array.append(assembler.occupancy(i)) 
#     technician_array.append(technician.occupancy(i))
# #     print(assembler.occupancy[(100*i):(100*(i+1))].mean())
# #     print(technician.occupancy[(100*i):(100*(i+1))].mean())
# #     print(production_control.occupancy[(100*i):(100*(i+1))].mean())
# #     print(qual_inspector.occupancy[(100*i):(100*(i+1))].mean())
# # technician.print_statistics()
# # production_control.print_statistics()
# # qual_inspector.print_statistics()
# print("test")
# print(assembler_array[600])


<class 'numpy.ndarray'>
1000
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [138]:
# print("occupancy:")
# print(assembler.occupancy.mean())
# print(production_control.occupancy.mean())
# print(qual_inspector.occupancy.mean())
# print(technician.occupancy.mean())

# print("available_quantity:")
# print(assembler.available_quantity.mean())
# print(production_control.available_quantity.mean())
# print(qual_inspector.available_quantity.mean())
# print(technician.available_quantity.mean())

# print("off clock:")
# print(1 - assembler.available_quantity.mean() - assembler.occupancy.mean())
# print(1 - production_control.available_quantity.mean() - production_control.occupancy.mean())
# print(1 - qual_inspector.available_quantity.mean() - qual_inspector.occupancy.mean())
# print(1 - technician.available_quantity.mean() - technician.occupancy.mean())

# assembly_bench_1.queue.print_statistics()
# quality_bench_1.queue.print_statistics()
# forklift_1.queue.print_statistics()
# print(assembly_bench_1.state())
# print(assembly_bench_1.state())

NameError: name 'now' is not defined

Statistics of quality_bench_1_queue at      1000    
                                                                     all    excl.zero         zero
-------------------------------------------- -------------- ------------ ------------ ------------
Length of quality_bench_1_queue              duration           1000         1000            0    
                                             mean                  0            0    
                                             std.deviation         0            0    

                                             minimum               0            0    
                                             median                0            0    
                                             90% percentile        0            0    
                                             95% percentile        0            0    
                                             maximum               0            0    

Length of stay in quality_bench_1_queue      e